In [1]:
# https://zhuanlan.zhihu.com/p/335753926
import os, sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from collections import OrderedDict
import numpy as np
import cv2
import torch
from pytorchocr.base_ocr_v20 import BaseOCRV20

class PPOCRv3RecConverter(BaseOCRV20):
    def __init__(self, config, paddle_pretrained_model_path, **kwargs):
        para_state_dict, opti_state_dict = self.read_paddle_weights(paddle_pretrained_model_path)
        para_state_dict = self.del_invalid_state_dict(para_state_dict)
        total_params = sum(p.size if isinstance(p, np.ndarray) else p.numel() for p in para_state_dict.values())
        print(f'Total number of parameters in para_state_dict: {total_params}')
        out_channels = list(para_state_dict.values())[-1].shape[0]
        print('out_channels: ', out_channels)
        print(type(kwargs), kwargs)
        kwargs['out_channels'] = out_channels
        super(PPOCRv3RecConverter, self).__init__(config, **kwargs)
        # self.load_paddle_weights(paddle_pretrained_model_path)
        self.load_paddle_weights([para_state_dict, opti_state_dict])
        print('model is loaded: {}'.format(paddle_pretrained_model_path))
        otal_params = sum(p.numel() for p in self.net.parameters())
        print(f'Total number of parameters in self.net: {total_params}')
        self.net.eval()


    def del_invalid_state_dict(self, para_state_dict):
        new_state_dict = OrderedDict()
        for i, (k,v) in enumerate(para_state_dict.items()):
            if k.startswith('Teacher.'):
                continue

            elif k.startswith('Student.head.sar_head.'):
                continue

            else:
                new_state_dict[k] = v
        return new_state_dict


    def load_paddle_weights(self, paddle_weights):
        para_state_dict, opti_state_dict = paddle_weights
        [print('paddle: {} ---- {}'.format(k, v.shape)) for k, v in para_state_dict.items()]
        # [print('pytorch: {} ---- {}'.format(k, v.shape)) for k, v in self.net.state_dict().items()]

        for k,v in self.net.state_dict().items():
            if k.endswith('num_batches_tracked'):
                continue

            ppname = k
            ppname = ppname.replace('.running_mean', '._mean')
            ppname = ppname.replace('.running_var', '._variance')
            ppname = ppname.replace('neck.encoder.', 'head.ctc_encoder.encoder.')
            ppname = ppname.replace('head.fc.', 'head.ctc_head.fc.')

            try:
                try:
                    if ppname.endswith('fc1.weight') or ppname.endswith('fc2.weight') \
                            or ppname.endswith('fc.weight') or ppname.endswith('qkv.weight') \
                            or ppname.endswith('proj.weight') or ppname.endswith("MLM_VRM.MLM.w0_linear.weight") \
                                or ppname.endswith("MLM_VRM.MLM.we.weight") \
                                    or ppname.endswith("MLM_VRM.Prediction.pp.w0.weight") \
                                        or ppname.endswith("MLM_VRM.Prediction.pp.we.weight") or ("weight" in ppname and "MLM_VRM") \
                                            and ("backbone" not in ppname) \
                                                and ("MLM_SequenceModeling_mask" not in ppname) \
                                                    and ("MLM_SequenceModeling_WCL" not in ppname): # or ("weight" in ppname and "MLM_VRM")
                        # print("Transpose!")
                        self.net.state_dict()[k].copy_(torch.Tensor(para_state_dict[ppname].T))
                    else:
                        # print("No transpose")

                        self.net.state_dict()[k].copy_(torch.Tensor(para_state_dict[ppname]))
                except:
                    self.net.state_dict()[k].copy_(torch.Tensor(para_state_dict[ppname]))
            except Exception as e:
                print('pytorch: {}, {}'.format(k, v.size()))
                print('paddle: {}, {}'.format(ppname, para_state_dict[ppname].shape))
                raise e

        print('model is loaded.')


if __name__ == '__main__':
    import argparse, json, textwrap, sys, os

    # parser = argparse.ArgumentParser()
    # parser.add_argument("--src_model_path", type=str, help='Assign the paddleOCR trained model(best_accuracy)')
    # args = parser.parse_args()
    src_model_path = "/raid/kientdt/shared_drive_cv/ocr/kientdt/PaddleOCR/output/rec/vi_r45_visionlan_19_2"
    cfg = {'model_type':'rec',
           'algorithm':'VisionLAN',
           'Transform':None,
           'Backbone':{'name':'ResNet45',
                       'strides': [2, 2, 2, 1, 1]},
           'Head':{'name':'VLHead',
            "n_layers": 3,
            "n_position": 256,
            "n_dim": 512,
            "max_text_length": 40,
            "training_step": 'LA'}
           }
    paddle_pretrained_model_path = os.path.join(os.path.abspath(src_model_path), 'best_accuracy')
    converter = PPOCRv3RecConverter(cfg, paddle_pretrained_model_path)
    save_basename = os.path.basename(os.path.abspath(src_model_path))

    save_name = 'visionlan_ver1.pth'
    converter.save_pytorch_weights(save_name)
    print('done.')
    print(converter.net)



####Model path: /raid/kientdt/shared_drive_cv/ocr/kientdt/PaddleOCR/output/rec/r45_visionlan

Total number of parameters in para_state_dict: 33475199
out_channels:  230
<class 'dict'> {}
n_dim: 512, n_position: 256, n_max_char: 41
n_dim: 512, n_position: 256, n_max_char: 41
paddle: backbone.conv1.weight ---- (32, 3, 3, 3)
paddle: backbone.bn1.weight ---- (32,)
paddle: backbone.bn1.bias ---- (32,)
paddle: backbone.bn1._mean ---- (32,)
paddle: backbone.bn1._variance ---- (32,)
paddle: backbone.layer1.0.conv1.weight ---- (32, 32, 1, 1)
paddle: backbone.layer1.0.bn1.weight ---- (32,)
paddle: backbone.layer1.0.bn1.bias ---- (32,)
paddle: backbone.layer1.0.bn1._mean ---- (32,)
paddle: backbone.layer1.0.bn1._variance ---- (32,)
paddle: backbone.layer1.0.conv2.weight ---- (32, 32, 3, 3)
paddle: backbone.layer1.0.bn2.weight ---- (32,)
paddle: backbone.layer1.0.bn2.bias ---- (32,)
paddle: backbone.layer1.0.bn2._mean ---- (32,)
paddle: backbone.layer1.0.bn2._variance ---- (32,)
paddle: backbone.layer1.0.downsample.0.weight ---- (32, 32, 1, 1)
paddle: backbone.layer1.0.downsample.1.weight 

In [2]:
model = converter.net

In [3]:
model

BaseModel(
  (backbone): ResNet45(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): BasicBlock(
        (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride

In [4]:
# import cv2
# import numpy as np

# # Load the image
# image_path = '/raid/kientdt/shared_drive_cv/ocr/kientdt/PaddleOCR2Pytorch/img_169.jpg'
# image = cv2.imread(image_path)

# # Convert the image to RGB (if it's in BGR)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# # Resize the image to the expected input size (e.g., 32x100)
# image_resized = cv2.resize(image, (100, 32))

# # Normalize the image
# image_normalized = image_resized / 255.0

# # Convert to tensor and add batch dimension
# image_tensor = torch.tensor(image_normalized).permute(2, 0, 1).unsqueeze(0).float()


In [5]:
from pytorchocr.postprocess.rec_postprocess import VLLabelDecode
from torchvision import transforms
from PIL import Image
decoder = VLLabelDecode("/raid/kientdt/shared_drive_cv/ocr/kientdt/PaddleOCR/ppocr/utils/dict/my_vi_dict.txt", False, max_text_length = 40)

In [6]:
image_path = "/raid/kientdt/shared_drive_cv/ocr/hieunq/PDFs_130125/TNR_Bold/train/img_9646.jpg"
Image.open(image_path)

In [7]:

if True:
    # image_path = '/raid/kientdt/shared_drive_cv/ocr/kientdt/PaddleOCR2Pytorch/img_169.jpg'
    # output_txt = './demo/predictions.txt'
    # image_list = os.listdir(image_path)
    img_width = 256
    img_height = 64
    transf = transforms.ToTensor()
    # test_acc_counter = Attention_AR_counter('\ntest accuracy: ', "/raid/kientdt/shared_drive_cv/ocr/kientdt/PaddleOCR/ppocr/utils/dict/my_vi_dict.txt",
    #                                         False)

    model.eval()
    if True:
        img = Image.open(image_path).convert('RGB')
        img = img.resize((img_width, img_height))
        img = transf(img)
        img = torch.unsqueeze(img,dim = 0)
        target = ''
        output = model(img)
        # pre_string = test_acc_counter.convert(output, 5)
        # print('pre_string:',pre_string[0])
        # with open(output_txt,'a') as f:
            # f.write(img_name+':'+pre_string[0]+'\n')

feat: torch.Size([1, 512, 8, 32])
target: None
x: torch.Size([1, 256, 512])
1.enc_output at transformer: torch.Size([1, 256, 512])
enc_output at transformer: torch.Size([1, 256, 512])
contextual_feature: torch.Size([1, 256, 512])
cnn_feature: torch.Size([1, 256, 512])
reading_order: torch.Size([1, 41, 512])
enc_output: torch.Size([1, 256, 512])
self.wv: Linear(in_features=512, out_features=512, bias=True), self.w0: Linear(in_features=41, out_features=256, bias=True)
t.permute(0,2,1): torch.Size([1, 256, 512]), self.wv(enc_output): torch.Size([1, 256, 512])


In [8]:
decoder(output)

RuntimeError: The expanded size of the tensor (229) must match the existing size (230) at non-singleton dimension 1.  Target sizes: [1, 229].  Tensor sizes: [230]

In [ ]:
output

(tensor([[[ 2.1689, -3.8158, -5.5368,  ...,  4.0971,  0.1235, -1.4682]],
 
         [[-0.4243, -0.3389, -0.9833,  ...,  6.4294,  3.0310, -4.8024]],
 
         [[ 3.9689, -1.9176, -4.6427,  ...,  4.8496, -1.2137, -2.9893]],
 
         ...,
 
         [[ 1.8408, -0.7990, -2.8129,  ...,  7.4797,  0.8752, -3.5223]],
 
         [[ 3.3813, -0.3652, -3.3112,  ...,  6.2396,  0.0144, -3.6870]],
 
         [[ 2.4544, -0.6290, -3.0637,  ...,  6.4031,  0.6885, -3.9720]]],
        grad_fn=<TransposeBackward0>),
 tensor([[[1.0607, 0.0000, 0.0591,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0370, 0.0000,  ..., 0.0000, 0.0000, 0.9042],
          [0.0000, 0.0000, 0.0000,  ..., 0.7529, 0.0000, 1.2725],
          ...,
          [0.0000, 1.7642, 0.0000,  ..., 0.0000, 2.3638, 0.0000],
          [0.0000, 3.0241, 0.0000,  ..., 0.0000, 3.1049, 0.0000],
          [3.6165, 0.3620, 3.1657,  ..., 0.0000, 0.7767, 0.3339]]],
        grad_fn=<TransposeBackward0>))